<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3%E3%82%B5%E3%83%BC%E3%83%81%E3%81%AERDF%E3%82%B9%E3%83%88%E3%82%A2%E3%82%92%E5%AF%BE%E8%B1%A1%E3%81%97%E3%81%9FSPARQL%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ジャパンサーチのRDFストアを対象したSPARQLチュートリアル

ジャパンサーチのRDFストアを対象したSPARQLのチュートリアルです。

参考：https://jpsearch.go.jp/static/developer/ja.html

## 初期設定

In [ ]:
# 日本語の文字化け対策
!pip install japanize-matplotlib
import requests
import urllib
import json
import pprint
import matplotlib.pyplot as plt
import japanize_matplotlib

endpoint = "https://jpsearch.go.jp/rdf/sparql/"

def execQuery(q):
  url = "{}?query={}&format=json&output=json&results=json".format(endpoint, urllib.parse.quote(q))
  return requests.get(url).json()

def displayGraph(X, Y, title, xlabel, ylabel):
  # グラフの大きさ指定
  plt.figure(figsize=(10, 10))

  # 棒グラフを表示
  plt.barh(X, Y)

  # XとYのラベル
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

  # タイトル表示
  plt.title(title, fontsize=15)

## 例1:作品数の多い順に作者を取得する

### クエリ例

In [ ]:
q = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX chname: <https://jpsearch.go.jp/entity/chname/>
SELECT distinct (count(?cho) as ?count) ?label WHERE {
    ?cho schema:creator ?creator .
    ?creator rdfs:label ?label . 
    ?creator rdfs:isDefinedBy chname: # 正規化されている名前のみ

    # 以下のように書くことができます。

    # ?cho schema:creator ?creator .
    # ?creator rdfs:label ?label; rdfs:isDefinedBy chname: # 正規化されている名前のみ
}
ORDER BY DESC(?count)
LIMIT 20
"""

results = execQuery(q)
for obj in results["results"]["bindings"]:
  # pprint.pprint(obj)
  count = obj["count"]["value"]
  label = obj["label"]["value"]
  print(label, "\t", count)

### 可視化例

In [ ]:
x_list = []
x_value = "label"
y_list = []
y_value = "count"
for obj in results["results"]["bindings"]:
  x_list.append(obj[x_value]["value"])  
  y_list.append(int(obj[y_value]["value"]))

displayGraph(x_list, y_list, "作品数の多い作者", "作品数", "作者", )  

## 例2:位置情報を取得する

### クエリ例

In [ ]:
q = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX chname: <https://jpsearch.go.jp/entity/chname/>
SELECT distinct ?s ?label ?lat ?long WHERE {
    # ?s rdfs:label ?label .
    # ?s schema:geo ?geo . 
    # ?geo schema:latitude ?lat . 
    # ?geo schema:longitude ?long . 

    # 以下のように書くことができます。

    ?s rdfs:label ?label; schema:geo ?geo . 
    ?geo schema:latitude ?lat; schema:longitude ?long . 
}
"""

results = execQuery(q)

### 可視化例

In [ ]:
# 結果をjson形式で保存
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

In [ ]:
# 結果をtable形式で表示
from pandas import json_normalize

# 変換用の関数
def convertSparqlResult2Df(results, mappings):
  data = []

  for obj in results["results"]["bindings"]:
    item = {}

    for mapping in mappings:
      label = mapping["label"]
      query_var = mapping["query_var"]
      item[label] = obj[query_var]["value"]

    data.append(item)

  df = json_normalize(data)
  return df

mappings = [
      {
        "label": "ラベル",
        "query_var" : "label"
      },
      {
        "label": "緯度",
        "query_var" : "lat"
      },
      {
        "label": "経度",
        "query_var" : "long"
      }
  ]

df = convertSparqlResult2Df(results, mappings)
df

#### 地図表示

In [ ]:
!pip install folium
import folium
from folium import plugins

In [ ]:
# 初期値
m = folium.Map(location=[35.681382, 139.76608399999998], zoom_start=2) #東京駅の緯度経度

# クラスタ
marker_cluster = plugins.MarkerCluster().add_to(m)

# マーカーの追加
for index, data in df.iterrows():
  popup=folium.Popup(data["ラベル"], max_width=200)
  folium.Marker(location=[data["緯度"], data["経度"]], popup=popup).add_to(marker_cluster)

# 表示 
m

## 例3:IIIFコレクションを作成・表示する

### データ取得

In [ ]:
# 変数

## ページの上限。全てで実行したい場合には、-1を与えてください。時間がかかります。
thres = 5 # -1

url_prefix = ""

##  以下、関数と実行

import os

def getManifestsByAttribution():

  page = 0

  unit = 10000

  manifestsByAttribution = {}

  while 1:
      print("page", page)

      q = f"""
        SELECT DISTINCT ?url ?label ?provider_label WHERE {{
        ?s rdfs:label ?label; jps:accessInfo ?accessInfo . 
        ?accessInfo schema:url ?url; schema:provider/rdfs:label ?provider_label .
        ?url rdf:type <http://iiif.io/api/presentation/2#Manifest> .
      }} limit {str(unit)} offset {str(unit * page)}
      """

      results = execQuery(q)

      if len(results["results"]["bindings"]) == 0 or (thres > 0 and thres < page):
          break

      page += 1

      for obj in results["results"]["bindings"]:
          manifest = obj["url"]["value"]
          label = obj["label"]["value"]
          attribution = obj["provider_label"]["value"]

          if attribution not in manifestsByAttribution:
              manifestsByAttribution[attribution] = []

          manifestsByAttribution[attribution].append({
              "@id": manifest,
              "@type": "sc:Manifest",
              "label": label
          })

  return manifestsByAttribution

def createCollection(manifestsByAttribution):
  collections = []

  collection = {
      "@context": "http://iiif.io/api/presentation/2/context.json",
      "@id": f"{url_prefix}/collection.json",
      "@type": "sc:Collection",
      "label": "Japan Search IIIF Collection",
      "collections": collections
  }

  for attribution in manifestsByAttribution:
      manifests = manifestsByAttribution[attribution]
      
      # コレクションを保存
      collectionByAttribution = {
          "@context": "http://iiif.io/api/presentation/2/context.json",
          "@id": f"{url_prefix}/collections/{attribution}.json",
          "@type": "sc:Collection",
          "label": f"{attribution}",
          "manifests": manifests
      }

      collection_path = f"collections/{attribution}.json"
      os.makedirs(os.path.dirname(collection_path), exist_ok=True)

      with open(collection_path, 'w', encoding='utf-8') as f:
        json.dump(collectionByAttribution, f, ensure_ascii=False, indent=2)

      # collectionsへ登録
      c = {
          "@id": collectionByAttribution["@id"],
          "@type": "sc:Collection",
          "label": f"{attribution} ({len(manifests)}件)"
      }
      collections.append(c)

  return collection

manifestsByAttribution = getManifestsByAttribution()

collection = createCollection(manifestsByAttribution)

with open("collection.json", 'w', encoding='utf-8') as f:
  json.dump(collection, f, ensure_ascii=False, indent=2)

### 可視化例（Mirador）

HTTPサーバーを立ち上げます。

In [ ]:
# サーバの起動
PORT = 8001
!nohup python3 -m http.server $PORT > server.log 2>&1 &

# Miradorのダウンロード
if not os.path.exists("index.html"):
  !wget http://mirador.cultural.jp/

# パス
mirador_path = "/index.html?manifest=collection.json"

# URLの表示
from google.colab import output
output.serve_kernel_port_as_window(PORT, path=mirador_path)